In [1]:
# Regression Week 2: Multiple Linear Regression Assignment 1
# Predicting House Prices (Multiple Variables)
import graphlab

In [3]:
# In this notebook you will use data on house sales in King County
# to predict prices using multiple regression. The first assignment
# will be about exploring multiple regression in particular exploring
# the impact of adding features to a regression and measuring error.
# In the second assignment you will implement a gradient descent algorithm.

In [4]:
# 1. If you are using SFrame, import graphlab and load in the house data,
# otherwise you can also download the csv. (Note that we will be using the
# training and testing csv files provided) e.g in python with SFrames:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to mikael.baymani@gmail.com and will expire on May 13, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1564423122.log


In [5]:
# 2. Split into training and test data Use this command to set the same seed
# for everyone: e.g. in python with SFrames:
train_data,test_data = sales.random_split(.8,seed=0)

In [10]:
# 3. Although we often think of multiple regression as including multiple different
# features (e.g. # of bedrooms, square feet, and # of bathrooms) but we can also consider
# transformations of existing variables e.g. the log of the square feet or even "interaction"
# variables such as the product of bedrooms and bathrooms. Add 4 new variables in both your
# train_data and test_data.

In [17]:
import math
train_data['bedrooms_squared'] = train_data['bedrooms']*train_data['bedrooms']
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
train_data['log_sqft_living'] = [math.log(x) for x in train_data['sqft_living']]
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']

In [18]:
train_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [19]:
test_data['bedrooms_squared'] = test_data['bedrooms']*test_data['bedrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']
test_data['log_sqft_living'] = [math.log(x) for x in test_data['sqft_living']]
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [20]:
# 4. Quiz Question: what are the mean (arithmetic average) values of your 4
# new variables on TEST data? (round to 2 digits)

In [26]:
print "%.2f" % test_data['bedrooms_squared'].mean()
print "%.2f" % test_data['bed_bath_rooms'].mean()
print "%.2f" % test_data['log_sqft_living'].mean()
print "%.2f" % test_data['lat_plus_long'].mean()

12.45
7.50
7.55
-74.65


In [25]:
# 5. Use graphlab.linear_regression.create (or any other regression library/function)
# to estimate the regression coefficients/weights for predicting ‘price’ for the following
# three models:(In all 3 models include an intercept -- most software does this by default).

In [27]:
model_1 = graphlab.linear_regression.create(train_data, target='price', features=['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long'], validation_set=None)
model_2 = graphlab.linear_regression.create(train_data, target='price', features=['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms'], validation_set=None)
model_3 = graphlab.linear_regression.create(train_data, target='price', features=['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long'], validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.057612     | 3193229.177894     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [30]:
# 6. Quiz Question: What is the sign (positive or negative) for the
# coefficient/weight for ‘bathrooms’ in Model 1?
model_1.coefficients

name,index,value,stderr
(intercept),None,-56140675.7444,1649985.42028
sqft_living,None,310.263325778,3.18882960408
bedrooms,None,-59577.1160682,2487.27977322
bathrooms,None,13811.8405418,3593.54213297
lat,None,629865.789485,13120.7100323
long,None,-214790.285186,13284.2851607


In [31]:
# 7. Quiz Question: What is the sign (positive or negative) for the
# coefficient/weight for ‘bathrooms’ in Model 2?
model_2.coefficients

name,index,value,stderr
(intercept),None,-54410676.1152,1650405.16541
sqft_living,None,304.449298057,3.20217535637
bedrooms,None,-116366.043231,4805.54966546
bathrooms,None,-77972.3305135,7565.05991091
lat,None,625433.834953,13058.3530972
long,None,-203958.60296,13268.1283711
bed_bath_rooms,None,26961.6249092,1956.36561555


In [33]:
# 8. Is the sign for the coefficient the same in both models?
# Think about why this might be the case.

In [34]:
# 9. Now using your three estimated models compute the RSS
# (Residual Sum of Squares) on the Training data.

In [35]:
RSS = lambda RMSE,n : n*RMSE*RMSE 

In [39]:
# 10. Quiz Question: Which model (1, 2 or 3) had the lowest RSS on
# TRAINING data?
print RSS(model_1.evaluate(train_data)['rmse'], len(train_data))
print RSS(model_2.evaluate(train_data)['rmse'], len(train_data))
print RSS(model_3.evaluate(train_data)['rmse'], len(train_data))

9.71328233544e+14
9.61592067856e+14
9.05276314555e+14


In [40]:
# 11. Now using your three estimated models compute the RSS on
# the Testing data
print RSS(model_1.evaluate(test_data)['rmse'], len(test_data))
print RSS(model_2.evaluate(test_data)['rmse'], len(test_data))
print RSS(model_3.evaluate(test_data)['rmse'], len(test_data))

2.26568089093e+14
2.24368799994e+14
2.51829318952e+14


In [41]:
# 12. Quiz Question: Which model (1, 2, or 3) had the lowest RSS on
# TESTING data?
# 13. Did you get the same answer for 9 and 11? Think about why
# this might be the case.